In [2]:
import pandas as pd

# SPRINT 1 — ETL (Data Cleaning & Preprocessing)
# Goal: Prepare clean and consistent customer dataset ready for further analysis.


In [61]:
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display

# 0) Paths
raw_path = Path("../Data/Raw/marketing_campaign.xlsx")
out_dir  = Path("../Data/Interim")
out_dir.mkdir(parents=True, exist_ok=True)

# 1) Load raw data 
df = pd.read_excel(raw_path)

# 2) Data type corrections
# Parse dates and normalize categorical values
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], errors="coerce")
for col in ["Education", "Marital_Status"]:
    df[col] = df[col].astype(str).str.strip().str.title()

# Handle missing values in Income using median imputation
df["Income"] = pd.to_numeric(df["Income"], errors="coerce")
df["Income"] = df["Income"].fillna(df["Income"].median())

# 3) Apply business logic filters
# Keep realistic age range for marketing use cases
df = df[(df["Year_Birth"] >= 1945) & (df["Year_Birth"] <= 2007)].copy()

# 4) Outlier handling
# Winsorization (1st–99th percentile) for spend and purchase-related features
spend_cols    = ["MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts","MntGoldProds"]
purchase_cols = ["NumDealsPurchases","NumWebPurchases","NumCatalogPurchases","NumStorePurchases"]
count_cols    = spend_cols + purchase_cols + ["NumWebVisitsMonth", "Income"]

for c in count_cols:
    if c in df.columns:
        lo, hi = df[c].quantile([0.01, 0.99])
        if pd.notna(lo) and pd.notna(hi):
            df[c] = df[c].clip(lo, hi)

# 5) Binary columns normalization
bin_cols = [c for c in df.columns if c.startswith("AcceptedCmp")] + ["Complain", "Response"]
for c in bin_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
    df[c] = np.where(df[c] > 0, 1, 0).astype(np.int8)

# 6) Quick QA checks
assert df["ID"].is_unique, "Są duplikaty ID – sprawdź dane."
assert df["Income"].isna().sum() == 0, "Income ma NaN po imputacji."
assert df["Dt_Customer"].isna().sum() == 0, "Dt_Customer ma NaN – sprawdź format dat."

# Validate numerical ranges
for c in ["Recency","NumWebVisitsMonth"] + purchase_cols + spend_cols:
    if c in df.columns:
        assert (df[c] >= 0).all(), f"Ujemne wartości w {c}"

# 7) Save clean interim dataset
clean_path = out_dir / "marketing_campaign_clean.csv"
df.to_csv(clean_path, index=False)

print("ETL gotowe")
print("Wymiary:", df.shape)
print("Zapisano do:", clean_path)
print("\nPodgląd:")
display(df.head())


ETL gotowe
Wymiary: (2221, 29)
Zapisano do: ..\Data\Interim\marketing_campaign_clean.csv

Podgląd:


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,Phd,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0
